In [1]:
import numpy as np


In [2]:
def logistic_function(x):
  return 1 / (1 + np.exp(-x))

In [3]:
def test_logistic_function():
    x_scalar = 0
    assert round(logistic_function(x_scalar), 3) == 0.5

    x_pos = 2
    assert round(logistic_function(x_pos), 3) == round(1 / (1 + np.exp(-2)), 3)

    x_neg = -3
    assert round(logistic_function(x_neg), 3) == round(1 / (1 + np.exp(3)), 3)

    x_array = np.array([0, 2, -3])
    expected = np.array([0.5, 0.881, 0.047])
    assert np.all(np.round(logistic_function(x_array), 3) == expected)

    print("✅ Sigmoid test passed")

test_logistic_function()


✅ Sigmoid test passed


In [4]:
def log_loss(y_true, y_pred):
    """
    Computes log loss for a single data point.
    """
    if y_pred == 0 or y_pred == 1:
        if (y_true == 1 and y_pred == 0) or (y_true == 0 and y_pred == 1):
            raise ValueError("log(0) encountered")
        return 0.0

    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))


In [5]:
def test_log_loss():
    assert log_loss(1, 1) == 0.0
    assert log_loss(0, 0) == 0.0

    try:
        log_loss(1, 0)
    except ValueError:
        pass

    try:
        log_loss(0, 1)
    except ValueError:
        pass

    assert np.isclose(log_loss(1, 0.8), -np.log(0.8))
    assert np.isclose(log_loss(0, 0.2), -np.log(0.8))

    print("✅ Log-loss test passed")

test_log_loss()


✅ Log-loss test passed


In [6]:
def cost_function(y_true, y_pred):

    n = len(y_true)
    loss = np.array([log_loss(y_true[i], y_pred[i]) for i in range(n)])
    return np.sum(loss) / n


In [7]:
def test_cost_function():
    y_true = np.array([1, 0, 1])
    y_pred = np.array([0.9, 0.1, 0.8])

    expected = (-(np.log(0.9)) - np.log(0.9) - np.log(0.8)) / 3
    assert np.isclose(cost_function(y_true, y_pred), expected)

    print("✅ Cost function test passed")

test_cost_function()


✅ Cost function test passed


In [8]:
def costfunction_logreg(X, y, w, b):
    z = np.dot(X, w) + b
    y_pred = logistic_function(z)
    return cost_function(y, y_pred)


In [9]:
X = np.array([[10, 20], [-10, 10]])
y = np.array([1, 0])
w = np.array([0.5, 1.5])
b = 1

print("Cost:", costfunction_logreg(X, y, w, b))


Cost: 5.500008350784906


In [10]:
def compute_gradient(X, y, w, b):
    n, d = X.shape
    z = np.dot(X, w) + b
    y_pred = logistic_function(z)

    grad_w = -(1 / n) * np.dot(X.T, (y - y_pred))
    grad_b = -(1 / n) * np.sum(y - y_pred)

    return grad_w, grad_b


In [11]:
grad_w, grad_b = compute_gradient(X, y, w, b)
print("grad_w:", grad_w)
print("grad_b:", grad_b)
print("✅ Gradient computation passed")


grad_w: [-4.99991649  4.99991649]
grad_b: 0.4999916492890759
✅ Gradient computation passed


In [12]:
def gradient_descent(X, y, w, b, alpha, n_iter, show_cost=False, show_params=False):

    cost_history = []
    params_history = []

    for i in range(n_iter):
        grad_w, grad_b = compute_gradient(X, y, w, b)

        w -= alpha * grad_w
        b -= alpha * grad_b

        cost = costfunction_logreg(X, y, w, b)

        cost_history.append(cost)
        params_history.append((w.copy(), b))

        if show_cost and (i % 100 == 0):
            print(f"Iter {i}: Cost = {cost:.6f}")

    return w, b, cost_history, params_history


In [13]:
X_test = np.array([[0.1, 0.2], [-0.1, 0.1]])
y_test = np.array([1, 0])

w = np.zeros(2)
b = 0.0

w_out, b_out, cost_history, _ = gradient_descent(X_test, y_test, w, b, 0.1, 100)

assert cost_history[-1] < cost_history[0]
print("✅ Gradient descent test passed")


✅ Gradient descent test passed


In [14]:
def prediction(X, w, b, threshold=0.5):
    z = np.dot(X, w) + b
    y_prob = logistic_function(z)
    return (y_prob >= threshold).astype(int)


In [15]:
X_test = np.array([[0.5, 1.0], [1.5, -0.5], [-0.5, -1.0]])
w_test = np.array([1.0, -1.0])
b_test = 0.0

expected = np.array([0, 1, 1])
assert np.array_equal(prediction(X_test, w_test, b_test), expected)

print("✅ Prediction test passed")


✅ Prediction test passed


In [16]:
def evaluate_classification(y_true, y_pred):

    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))

    confusion_matrix = np.array([[TN, FP],
                                  [FN, TP]])

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return confusion_matrix, precision, recall, f1


In [17]:
y_true = np.array([1, 0, 1, 0])
y_pred = np.array([1, 0, 0, 0])

cm, p, r, f1 = evaluate_classification(y_true, y_pred)
print("Confusion Matrix:\n", cm)
print("Precision:", p, "Recall:", r, "F1:", f1)
print("✅ Evaluation test passed")


Confusion Matrix:
 [[2 0]
 [1 1]]
Precision: 1.0 Recall: 0.5 F1: 0.6666666666666666
✅ Evaluation test passed
